In [2]:
from bokeh.plotting import figure
from bokeh.layouts import row, column, widgetbox, gridplot

from bokeh.io import curdoc
from bokeh.io import output_notebook, show, output_file

from bokeh.models import ColumnDataSource, HoverTool, TapTool, Range1d, OpenURL
from bokeh.models import LinearColorMapper , ColorBar
from bokeh.models.widgets import Select, Slider


import numpy as np

In [3]:
def palette(name_of_mpl_palette):
    """ Transforms a matplotlib palettes into a bokeh 
    palettes
    """
    from matplotlib.colors import rgb2hex
    import matplotlib.cm as cm
    colormap =cm.get_cmap(name_of_mpl_palette) #choose any matplotlib colormap here
    bokehpalette = [rgb2hex(m) for m in colormap(np.arange(colormap.N))]
    return bokehpalette


my_palette = palette("viridis")


html_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">XSigma: </span>
            <span style="font-size: 13px; color: #515151">@z1</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">RA: </span>
            <span style="font-size: 13px; color: #515151;">@x1</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">DEC: </span>
            <span style="font-size: 13px; color: #515151;">@y1</span>
        </div>
    </div>
"""

In [4]:
selected_exposure = '00000003'#args['exposure']
selected_arm = 'r' #args['arm']
selected_spectrograph =  4 #args['spectrograph']

# determining the position of fibers:
c1,c2 = selected_spectrograph*500, (selected_spectrograph+1)*500
qlf_fiberid = np.arange(0,5000)[c1:c2] 
left, right, bottom, top =  331, 335, 13, 16.7

my_hover = HoverTool(tooltips=html_tooltip)

z1 = xwsigma['XSIGMA']
source = ColumnDataSource(data={
    'x1' : xwsigma['RA'][c1:c2],
    'y1' : xwsigma['DEC'][c1:c2],
    'z1' : xwsigma['XSIGMA'],
    'QLF_FIBERID': qlf_fiberid,
})

source_comp = ColumnDataSource(data = {
    'x1': xwsigma['RA'][:c1] + xwsigma['RA'][c2:],
    'y1': xwsigma['DEC'][:c1] + xwsigma['DEC'][c2:],
    'z1': ['']*4500
})

p = figure( title = 'XSIGMA', x_axis_label='RA', y_axis_label='DEC',
           plot_width=700, plot_height=600
           ,x_range=Range1d(left, right), y_range=Range1d(bottom, top)
          , tools= [my_hover, "pan,box_zoom,reset,crosshair"])

mapper = LinearColorMapper(palette= my_palette,
                           low=0.98*np.min(z1), 
                           high=1.02*np.max(z1))#nipy_spectral)#Viridis256)#,RdYlBu11)#Viridis256)#, low=0, high=100)


p.circle('x1','y1', source = source, name="data", radius = 0.016,
        fill_color={'field': 'z1', 'transform': mapper}, 
         line_color='black', line_width=0.1)

p.circle('x1','y1', source = source_comp, radius = 0.015,
         fill_color = 'lightgray', line_color='black', line_width=0.3)

# bokeh.pydata.org/en/latest/docs/reference/models/annotations.html
color_bar = ColorBar(color_mapper=mapper, label_standoff=-13,
                     major_label_text_font_style="bold", padding = 26,
                     major_label_text_align='right',
                     major_label_text_font_size="10pt",
                     location=(0, 0))

p.add_layout(color_bar, 'left')
#layout = row([color_bar, p])
output_notebook()
#output_file('output_XSigma.html')
show(p)  

Loading BokehJS ...

### Spectrum

In [5]:
skr_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Wavelength: </span>
            <span style="font-size: 13px; color: #515151">@wl &#8491</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">y: </span>
            <span style="font-size: 13px; color: #515151;">@med_resid</span>
        </div>
    </div>
"""

wavg_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Wavelength: </span>
            <span style="font-size: 13px; color: #515151">@wl &#8491</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">y: </span>
            <span style="font-size: 13px; color: #515151;">@wavg_resid</span>
        </div>
    </div>
"""

skr_hover=HoverTool(tooltips=skr_tooltip, mode='vline')
wavg_hover=HoverTool(tooltips=wavg_tooltip, mode='vline')


skyres_source = ColumnDataSource(
                data={'wl': skyresid['WAVELENGTH'],
                      'med_resid' : skyresid['MED_RESID_WAVE'],
                      'wavg_resid':  skyresid['WAVG_RES_WAVE']
                     })

p1 = figure(title= 'MED_RESID_WAVE', 
            x_axis_label='Angstrom',
            plot_width = 720, plot_height = 240,
          tools=[skr_hover,"pan,box_zoom,reset,crosshair, lasso_select" ])
p1.line('wl', 'med_resid', source=skyres_source)

p2 = figure(title= 'WAVG_RESID_WAVE', 
            x_axis_label='Angstrom',
            plot_width = 720, plot_height = 240,
          tools=[wavg_hover,"pan,box_zoom,reset,crosshair, lasso_select" ])
p2.line('wl', 'wavg_resid', source=skyres_source)

p1.x_range = p2.x_range
layout=column(p1,p2)

# output_file('output_res_spectrum1.html')
show(layout)


In [6]:
skr_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Wavelength: </span>
            <span style="font-size: 13px; color: #515151">@wl &#8491</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">y: </span>
            <span style="font-size: 13px; color: #515151;">@med_resid</span>
        </div>
    </div>
"""

wavg_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Wavelength: </span>
            <span style="font-size: 13px; color: #515151">@wl &#8491</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">y: </span>
            <span style="font-size: 13px; color: #515151;">@wavg_resid</span>
        </div>
    </div>
""" 
skr_hover=HoverTool(tooltips=skr_tooltip)
wavg_hover=HoverTool(tooltips=wavg_tooltip)


skyres_source = ColumnDataSource(
                data={'wl': skyresid['WAVELENGTH'],
                      'med_resid' : skyresid['MED_RESID_WAVE'],
                      'wavg_resid':  skyresid['WAVG_RES_WAVE'],
                       'wd': [1] *len(skyresid['WAVELENGTH']),
                      'bottom':[0.] *len(skyresid['WAVELENGTH'])
                     })

p1 = figure(title= 'MED_RESID_WAVE', 
            x_axis_label='Angstrom',
            plot_width = 720, plot_height = 240,
          tools=[skr_hover,"pan,box_zoom,reset, crosshair" ])

p1.vbar(x='wl',width='wd', top='med_resid',  bottom='bottom', source=skyres_source
       , hover_color='red')

p2 = figure(title= 'WAVG_RESID_WAVE', 
            x_axis_label='Angstrom',
            plot_width = 720, plot_height = 240,
          tools=[wavg_hover,"pan,box_zoom,reset, crosshair"])

p2.vbar(x='wl',width='wd', top='wavg_resid',  bottom='bottom', source=skyres_source
       ,hover_color='red')

p1.x_range = p2.x_range
layout=column(p1,p2)

#output_file('output_res_spectrum2.html')
show(layout)

In [10]:
skr_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Wavelength: </span>
            <span style="font-size: 13px; color: #515151">@wl &#8491</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">y: </span>
            <span style="font-size: 13px; color: #515151;">@med_resid</span>
        </div>
    </div>
"""

wavg_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Wavelength: </span>
            <span style="font-size: 13px; color: #515151">@wl &#8491</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">y: </span>
            <span style="font-size: 13px; color: #515151;">@wavg_resid</span>
        </div>
    </div>
"""

skr_hover=HoverTool(tooltips=skr_tooltip, mode='vline')
wavg_hover=HoverTool(tooltips=wavg_tooltip, mode='vline')


skyres_source = ColumnDataSource(
                data={'wl': skyresid['WAVELENGTH'],
                      'med_resid' : skyresid['MED_RESID_WAVE'],
                      'wavg_resid':  skyresid['WAVG_RES_WAVE']
                     })

p1 = figure(title= 'MED_RESID_WAVE', 
            x_axis_label='Angstrom',
            plot_width = 720, plot_height = 340,
          tools=[skr_hover,"pan,box_zoom,reset,crosshair, lasso_select" ])

p1.line('wl', 'med_resid', source=skyres_source)

p1.circle('wl', 'med_resid', source=skyres_source, alpha = 0, size=1,
          hover_alpha=1,
         hover_fill_color='orange', hover_line_color='red') 


p2 = figure(title= 'WAVG_RESID_WAVE', 
            x_axis_label='Angstrom',
            plot_width = 720, plot_height = 340,
          tools=[wavg_hover,"pan,box_zoom,reset,crosshair, lasso_select" ])

p2.line('wl', 'wavg_resid', source=skyres_source)
p2.circle('wl', 'wavg_resid', source=skyres_source, alpha=0, size=1,
         hover_alpha=1,
          hover_fill_color='orange', hover_line_color='red') 



p1.x_range = p2.x_range
layout=column(p1,p2)
#output_file('output_res_spectrum3.html')
show(layout)



In [1]:
# Pre Load this!
selected_exposure = '00000003'#args['exposure']
selected_arm = 'r' #args['arm']
selected_spectrograph =  4 #args['spectrograph']

from scalar_metrics_v02 import LoadMetrics

cam = selected_arm+str(selected_spectrograph)
exp, night = '00000003', '20190101'
lm = LoadMetrics(cam, exp, night);
metrics, tests  = lm.metrics, lm.tests 

countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

countpix loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countpix-r4-00000003.yaml
getbias loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getbias-r4-00000003.yaml
getrms loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getrms-r4-00000003.yaml
xwsigma loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-xwsigma-r4-00000003.yaml
countbins loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countbins-r4-00000003.yaml
integ loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-integ-r4-00000003.yaml
skycont loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skycont-r4-00000003.yaml
skypeak loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skypeak-r4-00000003.yaml
skyresid loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skyresid-r4-00000003.yaml

### OLD

In [ ]:
import numpy as np
from bokeh.plotting import output_notebook

In [ ]:
from bokeh.plotting import figure, show
#from bokeh.plotting import output_file
from bokeh.plotting import output_notebook
import numpy as np
# from bokeh.io import output_file # Outra aba

import getpass
usr = getpass.getuser()
print( usr)
import datetime

In [ ]:
import importlib
import scalar_metrics_v02
importlib.reload(scalar_metrics_v02)
from scalar_metrics_v02 import LoadMetrics

In [ ]:
cam, exp, night = 'r3', '00000003', '20190101'

print ("\n\nTests for the available scalars and using yaml output\n", "="*50)
print ("TO DO:\n *docs of functions \n *what more else?")

lm = LoadMetrics(cam, exp, night);

In [ ]:
metrics, tests  = lm.metrics, lm.tests # Load_metrics_n_tests()

In [ ]:
countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

In [ ]:
from bokeh.models import ColumnDataSource
import bokeh.plotting as bk
bk.output_notebook()

In [ ]:
# Creating my new bokeh colormap palette

def palette(name_of_mpl_palette):
    """ Transforms a matplotlib palettes into a bokeh 
    palettes
    """
    from matplotlib.colors import rgb2hex
    import matplotlib.cm as cm
    colormap =cm.get_cmap(name_of_mpl_palette) #choose any matplotlib colormap here
    bokehpalette = [rgb2hex(m) for m in colormap(np.arange(colormap.N))]
    return bokehpalette

import bokeh.palettes as bp

cmap_all = bp.__palettes__

In [ ]:
maps = ["viridis","coolwarm_r","RdYlBu_r", "bwr","BrBG_r", "jet"
       , "cool", "spring", "winter"]
my_palette = palette("viridis")
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
'#999999', '#e41a1c', '#dede00']
my_palette = palette("BrBG_r")
my_palette = palette(maps[8])
my_palette = CB_color_cycle

## XWSIGMA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
from bokeh.models import Range1d
from bokeh.models import ColumnDataSource, HoverTool, TapTool #, OpenURL
from bokeh.models import LinearColorMapper ,    ColorBar
#from bokeh.palettes import (Colorblind3,RdYlBu11, YlGnBu9,Spectral3, 
#                    Plasma256, RdYlBu,Colorblind, Viridis3, Viridis256)


camnum =  [ int(s) for s in cam if s.isdigit()][0]
c1,c2  = camnum*500, (camnum+1)*500
qlf_fiberid = np.arange(0,5000)[c1:c2] # Evaluate inside QLF

z = np.array(xwsigma['WSIGMA'])

#colors = [
#    "#%02x%02x%02x" % (int(r), int(g), int(b)) 
#    for r, g, b, _ in 255*mpl.cm.nipy_spectral(mpl.colors.Normalize()(z))
#]

source = ColumnDataSource(data={
    'x1' : xwsigma['RA'][c1:c2],
    'y1' : xwsigma['DEC'][c1:c2],
    'z'  : z,
    'QLF_FIBERID': qlf_fiberid,
})

source_comp = ColumnDataSource(data = {
    'x_comp': xwsigma['RA'][:c1] + xwsigma['RA'][c2:],
    'y_comp': xwsigma['DEC'][:c1] + xwsigma['DEC'][c2:],   
})

left, right, bottom, top =  331, 335, 13, 16.7
hover = HoverTool(tooltips=[
         ("FIBER ID", "@QLF_FIBERID"),
         ("RA", "$x"),
         ("DEC", "$y"),
         ("WSIGMA", "@z"),
          ("fc","$color[ swatch]:fill_color")
         ])

#hover = HoverTool(names=["data"]) 
p = figure( plot_width=700, plot_height=600
           ,x_range=Range1d(left, right), y_range=Range1d(bottom, top)
          , tools= [hover, "pan,box_zoom,reset"])

mapper = LinearColorMapper(palette= my_palette, low=0.98*np.min(z), high=1.02*np.max(z))#nipy_spectral)#Viridis256)#,RdYlBu11)#Viridis256)#, low=0, high=100)


p.circle('x1','y1', source = source, name="data", radius = 0.015,
        fill_color={'field': 'z', 'transform': mapper}, # line_color=None,
         line_color='black', line_width=0.1)#""")# , source=source)

p.circle('x_comp','y_comp', source = source_comp, radius = 0.015, fill_color = 'lightgray'
          , line_color='black', line_width=0.3)#""")# , source=source)

# bokeh.pydata.org/en/latest/docs/reference/models/annotations.html
color_bar = ColorBar(color_mapper=mapper, label_standoff=-14,
                    major_label_text_font_style="bold", padding = 25,
                      major_label_text_align ='right',
                     major_label_text_font_size="10pt",
                     location=(0, 0))

p.add_layout(color_bar, 'left')

""",# radius=radii/radii,
fill_color=colors, fill_alpha=1.,
line_color=None)"""


show(p)  

In [ ]:
plt.scatter(xwsigma['RA'][c1:c2], xwsigma['DEC'][c1:c2], marker='.', c= xwsigma['XSIGMA'])#, c= mym['XSIGMA'])#, mym['WSIGMA'], edgecolor = 'k')
plt.colorbar()
plt.show()
plt.figure(figsize=(10,8))
plt.xlim(331,335)
plt.ylim(13,16.7)
plt.scatter(xwsigma['RA'][c1:c2], xwsigma['DEC'][c1:c2], marker='.', c= xwsigma['WSIGMA'])
plt.colorbar()



In [ ]:
qlf_fiberid = np.arange(0,5000)[c1:c2]
print (qlf_fiberid[0:10])

​

## AUXILIAR

In [ ]:
import numpy as np

from bokeh.io import show
from bokeh.layouts import gridplot
from bokeh.models import (
    ColumnDataSource,
    ColorBar,
    LinearColorMapper,
    LogColorMapper,
)

from bokeh.palettes import Viridis3, Viridis256
from bokeh.plotting import figure

x = np.random.random(2500) * 140 - 20
y = np.random.normal(size=2500) * 2 + 5

source = ColumnDataSource(dict(x=x, y=y))
opts = dict(x='x', line_color=None, source=source)


cmap_list = [Category10_10,Accent8,BuPu5,
                    Colorblind3,RdYlBu11, YlGnBu9,Spectral3, 
                    Plasma256, RdYlBu,Colorblind, Viridis3, Viridis256]

def make_plot(mapper, title):
    mapper.low_color = 'blue' # cor abaixo de low no range do LinearColorMapper
    mapper.high_color = 'red' # 
    p = figure(toolbar_location=None, tools='', title=title)
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0))
    p.circle(
        x='x', y='y',
        fill_color={'field': 'x', 'transform': mapper}, line_color=None,
        source=source
    )

    p.add_layout(color_bar, 'right')
    return p

p1 = make_plot(LinearColorMapper(palette=Viridis256, low=0, high=100), title='Viridis256 - Linear, low/high = blue/red')
p2 = make_plot(LogColorMapper(palette=Viridis256, low=0, high=100), title='Viridis256 - Log, low/high = blue/red')
p3 = make_plot(LinearColorMapper(palette=Viridis3, low=0, high=100), title='Viridis3 - Linear, low/high = blue/red')
p4 = make_plot(LogColorMapper(palette=Viridis3, low=0, high=100), title='Viridis3 - Log, low/high =, blue/red')

show(gridplot([p1, p2, p3, p4], ncols=2, plot_width=400, plot_height=300, toolbar_location=None))

In [ ]:
print( len(cmap_all))

for i in range(1,187):

    if 187%i == 0:

        print (i)

In [ ]:
p=[]
for i in range(10):
    p0 = make_plot(LinearColorMapper(palette=cmap_all[i], low=-20, high=120), title='%d'%(i))
    p.append(p0)
    
show(gridplot(p, ncols=2, plot_width=400, plot_height=300, toolbar_location=None))


In [ ]:
p =[]

cmap_list = [Category10_10, Accent8, BuPu5, Colorblind3
             ,RdYlBu11, YlGnBu9,Spectral3, Plasma256
             , RdYlBu11, Colorblind3, Viridis3, Viridis256]

j = 8

for i in range(j,j+4):
    p0 = make_plot(LinearColorMapper(palette=cmap_list[i], low=-20, high=120), title='%d'%(i))
    p.append(p0)

show(gridplot([p[0], p[1], p[2], p[3]], ncols=2, plot_width=400, plot_height=300, toolbar_location=None))

In [ ]:
from  astropy.io  import fits
import matplotlib.pyplot as plt
imf = '/home/felipe/quicklook/spectro/redux/exposures/20190101/00000004/sframe-r0-00000004.fits'
image_data = fits.getdata(imf, ext=0)
#print(fits.info(imf), '\n\n=====\n',fits.Header(imf))

In [ ]:
plt.figure(figsize =(18,4))
plt.imshow(image_data)
plt.show();plt.close()

In [ ]:
'''desi-00000003.fits.fz  fibermap-00000003.fits
desi-00000004.fits.fz  fibermap-00000004.fits
'''

imfolder = '/home/felipe/quicklook/spectro/data/20190101/'
imf  = imfolder+ 'fibermap-00000004.fits'
image_data = fits.getdata(imf, ext=0)

fits.info(imf)
#plt.figure(figsize =(10,10))
#plt.imshow(image_data)
#plt.show()
#plt.close()